In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./csvs/heart.csv")
df.sample(5)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
867,44,M,ASY,112,290,0,LVH,153,N,0.0,Up,1
577,67,M,ASY,146,369,0,Normal,110,Y,1.9,Flat,1
648,59,M,ASY,170,326,0,LVH,140,Y,3.4,Down,1
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0
107,34,M,ATA,150,214,0,ST,168,N,0.0,Up,0


In [3]:
col = ["Age" , "RestingBP" , "Cholesterol" , "FastingBS" , "MaxHR" , "Oldpeak"]
for i in col:
    df = df[(df[i] < (df[i].mean() + 3*df[i].std())) & (df[i] > (df[i].mean() - 3*df[i].std())) ]

In [6]:
df2 = df.copy()

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
col = [ "ChestPainType" , "RestingECG" ,  "ST_Slope"  ]
for i in col:
    df2[i] = LabelEncoder().fit_transform(df2[i])

In [9]:
df2 = pd.get_dummies(df2 , drop_first=True)
df2.sample(2)

,Age,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,Oldpeak,ST_Slope,HeartDisease,Sex_M,ExerciseAngina_Y
835,54,0,110,239,0,1,126,2.8,1,1,1,1
19,36,1,120,267,0,1,160,3.0,1,1,1,0


In [10]:
from sklearn.preprocessing import StandardScaler
st = StandardScaler()
X = st.fit_transform(df2.drop("HeartDisease" , axis=1))

In [21]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X , df2.HeartDisease , test_size=0.2)

In [22]:
from sklearn.svm import SVC
sc = SVC(kernel="linear")
sc.fit(X_train , y_train)

SVC(kernel='linear')

In [23]:
sc.score(X_test , y_test)

0.8944444444444445

In [26]:
from sklearn.model_selection import cross_val_score
cs = cross_val_score(SVC(kernel="linear") , X , df2.HeartDisease , cv=5)
cs

array([0.82222222, 0.8       , 0.85      , 0.84444444, 0.74301676])

In [27]:
cs.mean()

0.8119366852886406

In [36]:
from sklearn.ensemble import BaggingClassifier
bc_svm = BaggingClassifier(
    n_estimators=100,
    base_estimator=SVC(kernel="linear"),
    oob_score=True , 
    max_samples=0.8
)
bc_svm.fit(X_train , y_train)

BaggingClassifier(base_estimator=SVC(kernel='linear'), max_samples=0.8,
                  n_estimators=100, oob_score=True)

In [37]:
bc_svm.oob_score_

0.847009735744089

In [38]:
bc_svm.score(X_test , y_test)

0.8944444444444445

In [39]:
from sklearn.tree import DecisionTreeClassifier
dc = DecisionTreeClassifier()
dc.fit(X_train , y_train)

DecisionTreeClassifier()

In [40]:
dc.score(X_test , y_test)

0.7888888888888889

In [41]:
cv= cross_val_score(DecisionTreeClassifier() , X , df2.HeartDisease , cv=5)
cv.mean()

0.742917442582247

In [42]:
from sklearn.ensemble import  BaggingClassifier
bc_dtc = BaggingClassifier(
    n_estimators=100,
    base_estimator=DecisionTreeClassifier(),
    oob_score=True,
    max_samples=0.8
)
bc_dtc.fit(X_train , y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.8,
                  n_estimators=100, oob_score=True)

In [43]:
bc_dtc.oob_score_

0.8414464534075105

In [44]:
bc_dtc.score(X_test , y_test)

0.8888888888888888